## Computation of BoF with pyspark on EMR, loading from and storing on S3   

Here we show an example on how to load data from S3, here images, how to compute features of images with a MapReduce mindset, and how to store the dataframe containing the bag-of-feature on the S3

In [1]:
# install python librairies on the nodes
sc.install_pypi_package("pandas==1.0.4")
sc.install_pypi_package("s3fs")
sc.install_pypi_package("Pillow")
sc.install_pypi_package("python-resize-image")
sc.install_pypi_package("opencv-python")
sc.install_pypi_package("boto3")

#import python librairies
import numpy as np
import pandas as pd
import s3fs
from PIL import Image, ImageOps
from resizeimage import resizeimage
import cv2
import io
import os
import sys
import boto3

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1591068852364_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-1.0.4-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)


  Using cached Pillow-7.1.2-cp36-cp36m-manylinux1_x86_64.whl (2.1 MB)

  Using cached opencv_python-4.2.0.34-cp36-cp36m-manylinux1_x86_64.whl (28.2 MB)

  Using cached python_resize_image-1.1.19-py2.py3-none-any.whl (8.4 kB)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.9-py2.py3-none-any.whl (126 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached certifi-2020.4.5.1-py2.py3-none-any.whl (157 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)

Processing /mnt/var/lib/livy/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.1-cp36-cp36m-manylinux1_x86_64.whl (6.8 MB)
  Using cached joblib-0.15.1-py3-none-any.whl (298 kB)
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 

**Images are treated and resized and then ORB algorithm is used to extract the features in images**

In [2]:
s3_fs = s3fs.S3FileSystem()
image_path_list = s3_fs.ls('s3://XXXXXXXXXXXXX') # hidding the name of the bucket in the S3 for this public notebook
image_path_ini= ''

S3_path = 's3://XXXXXXXXXX' # hidding the name of the bucket in t the S3 for this public notebook
for img_pth in image_path_list:
    imgs_name = (img_pth.split("/")[2])
    image_path_ini += (S3_path + imgs_name + ',')
image_path = image_path_ini[:-1]

def img2catNdesc(image):
    try:
        # From bytes to image
        name, img = image
        image_img = Image.open(io.BytesIO(img))
        category = name.split('/')[-2]
        
        # Rescale image
        width = 100
        img_equ = ImageOps.equalize(image_img)
        img_full = resizeimage.resize_cover(img_equ, [width, width])
        arr_full = np.array(img_full)
        orbal = cv2.ORB_create(nfeatures = 50)
        kp, des = orbal.detectAndCompute(arr_full,None)
        tup_cat_des = ()
        cat_exp =''
        des_exp = []
        # initially one image gives one category and zero to several descriptors
        # the for loop generates tuples to set one cat per descriptor
        for ides in des:
            tup_cat_des = (category, ides)
        cat_exp, des_exp = tup_cat_des        
    except:
        arr_full = None
    return cat_exp, des_exp

# Takes all images and names
images =sc.binaryFiles(image_path, minPartitions = 4)
img_f1 = images.map(lambda img: img2catNdesc(img))
img_coll = img_f1.collect()

# example of the first tuple
img_coll[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Apple Braeburn', array([253, 245, 253, 231, 188, 111,  95, 188, 127, 242, 191,  28, 219,
       255,  89,  59, 191, 215, 125, 251, 249, 103, 251, 186, 251, 255,
       229,  60, 232, 251, 198, 114], dtype=uint8))

In [3]:
df = pd.DataFrame(data=img_coll, columns=['cate','descrip'])
cat_only = df[['cate']]

# changing list of array to features of df for each value in an array of the list
df_des_cells = pd.DataFrame(df['descrip'].values.tolist())
df_cat_des_cells = pd.concat([cat_only,df_des_cells],axis =1,ignore_index=True).dropna()
df_cat_des_cells.columns = df_cat_des_cells.columns.astype(str)
df_cat_des_cells.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                0    1    2    3    4    5  ...   27   28   29   30   31   32
0  Apple Braeburn  253  245  253  231  188  ...  229   60  232  251  198  114
1  Apple Braeburn  253  245  253  247  188  ...  229   60  252  251  198  114
2  Apple Braeburn  253  253  253  247  172  ...  229   52  232  251  198  114
3  Apple Braeburn  253  253  253  247  172  ...  229   52  233  251  198  114
4  Apple Braeburn  191   95  105  235  244  ...  167  196  191  191  208  241

[5 rows x 33 columns]

In [4]:
# moving to pyspark dataframe
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
spark_df = sqlContext.createDataFrame(df_cat_des_cells)

# vector assemblor geneates a column called "features" in the pyspark df and storing an array of the feature
# the "features" column will be used as input of the pyspark KMeans
from pyspark.ml.feature import VectorAssembler
input_list = df_cat_des_cells.columns.tolist()
# removing in the list, the column name 0 containing the categories 
# before assemblying the other feature values into an array in the column "features"
del input_list[0]

vecAssembler = VectorAssembler(inputCols=input_list, outputCol="features")
new_df = vecAssembler.transform(spark_df)

# clustering with 20 clusters which will be the references features for the bag of features (bof)
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=20, seed=1)  # choose 20 clusters 
model = kmeans.fit(new_df.select("features"))

# transformed variable is an input df with a new column called prediction (created and named by pyspark KMeans)
transformed = model.transform(new_df)

#CountVectorizer has a list of features as an input ex :[1, 9, 6, 6, 9, 8]
# first we take only the category column called "0" and "prediction" generate a rdd 
#then reducebykey to generate tuples containing the category and the features numbers for each category
rdd_features = transformed.select('0', 'prediction').rdd.map(lambda x:x).reduceByKey(lambda y,z: str(y) + ',' + str(z))

# reshaping the tuple as a tuple of a category and a list of features' numbers 
# then reducing to a dataframe
df_cat_feat= rdd_features.map(lambda tup: (tup[0], str(tup[1]).split(','))).toDF(['cate','words'])

from pyspark.ml.feature import CountVectorizer
# the dataframe column was named "words" on purpose since it is defined  as an input value in Countvectorizer
# Countvectorizer will comptue the bag of features based on the arrays in the column "words"
cv = CountVectorizer(inputCol="words", outputCol="bof")
cv_fitted =cv.fit(df_cat_feat)
df_cat_bof =cv_fitted.transform(df_cat_feat)
df_cat_bof.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+
|              cate|               words|                 bof|
+------------------+--------------------+--------------------+
|    Apple Braeburn|[13, 16, 13, 3, 8...|(20,[0,3,5,6,8,9,...|
|Apple Crimson Snow|[5, 10, 10, 10, 1...|(20,[0,1,2,3,4,7,...|
+------------------+--------------------+--------------------+

In [7]:
# changing to pandas df before storing
df_cat_bof_pd = df_cat_bof.toPandas()

from io import StringIO
import boto3
bucket = 'XXXXXXX' # hidding the name of the bucket for this public notebook
csv_buffer = StringIO()
df_cat_bof_pd.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'df_cat_bof_pd_fromEMR.csv').put(Body=csv_buffer.getvalue())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '9F4D8F7691F2F22B', 'HostId': 'ZhnYOWcQukqiCls204Qvnzpszc9ylR7CexjWQyDj+LzdKcaQgfvGirAnEhB4j+cJ/3YsRnnI9dQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ZhnYOWcQukqiCls204Qvnzpszc9ylR7CexjWQyDj+LzdKcaQgfvGirAnEhB4j+cJ/3YsRnnI9dQ=', 'x-amz-request-id': '9F4D8F7691F2F22B', 'date': 'Tue, 02 Jun 2020 06:24:18 GMT', 'etag': '"fca300601a9548fca537a579b23ada9e"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 1}, 'ETag': '"fca300601a9548fca537a579b23ada9e"'}

**Furework : Compute the bag of features with a CNN and perform clustering of the BoF computed by the ORB algorithm and the CNN.**